In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('./data/final_train.csv.zip', compression='zip')
#test = pd.read_csv('./data/final_test.csv.zip', compression='zip')
target = np.load('./data/target.npy')
#submission_ids = np.load('./data/submission_ids.npy')

In [3]:
target = np.log1p(target)
X_train, X_valid, Y_train, Y_valid = train_test_split(train, target, test_size=.15)


train_lgb = lgb.Dataset(data=X_train, label=Y_train)
valid_lgb = lgb.Dataset(data=X_valid, label=Y_valid)

In [ ]:
evaluation_results = {}

params = {
    'objective': 'regression_l2',
    'metric': 'rmse',
    'num_leaves': 40,
    'learning_rate': 0.005,
    'n_estimators': 10000,
    'num_trees': 10000,
    'bagging_fraction': 0.7,
    'bagging_freq': 5,
    'feature_fraction': 0.7,
    'max_depth': 20,
    'max_bin': 1200,
    'verbosity': -1,
    'nthread': 8
    }
params2 = { # control parameters
            'bagging_fraction': 0.7,
            'feature_fraction': 0.7,
            'lambda' : .1,
            'early_stopping_round' : 1000,
            # core parameters
            'task' : 'train',
            'objective' : 'regression',
            'boosting': 'gbdt',
            'num_leaves': 40,
            'learning_rate': 0.02,
           # Metric
           'metric': 'rmse',
            # other
            'max_depth': 10, 
           'num_trees': 10000
    }
model = lgb.train(
    params2,
    train_set=train_lgb,
    num_boost_round=1000,
    valid_sets=valid_lgb,
    early_stopping_rounds=3000,
    evals_result=evaluation_results,
    verbose_eval=200,
    )

In [ ]:
model_predictions_test = model.predict(test, num_iteration=model.best_iteration)
model_predictions_test

In [ ]:
model_predictions_test = np.expm1(model_predictions_test)
model_predictions_test

In [ ]:
dataset_submission = pd.DataFrame()
dataset_submission['ID'] = submission_ids
dataset_submission['target'] = model_predictions_test
dataset_submission.to_csv('./data/submission.csv', index=False)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as rmse